In [1]:
import re
import json
from functools import reduce

import pandas as pd

# Importing data

In [2]:
resumes = pd.read_csv("./data/resume_to_hackaton.csv", delimiter=";")
vacancies = pd.read_csv("./data/vacancies_to_hackaton.csv", delimiter=";")

vacancies.drop_duplicates(subset=['id'], inplace=True)

In [3]:
display(resumes)

,uuid,position,organization,description,start,end
0,e08588cc-4ad1-45be-ae95-fa3fd31bc32e,заведующая кафе-кладовщик,ООО Мираж,Составление ттк\r Проведение инвентаризаций\r ...,2012-04,2012-11
1,36c13102-4a62-477b-a475-f9a5b1c48218,Заведующий складом,Оконный завод,руководство складом и цехом по производству ко...,2014-02,2015-09
2,36c13102-4a62-477b-a475-f9a5b1c48218,старший кладовщик,"ООО "" Керамика сервис""",Полное ведение складского учета отдельно стоящ...,2012-03,2013-06
3,36c13102-4a62-477b-a475-f9a5b1c48218,бухгалтер складского учета,"Многопрофильный холдинг ""Вероника"" СПБ","Cклад, бухгалтер по учету ТМЦ и ОС, проведение...",2010-12,2011-05
4,85bfe532-1108-4142-bae5-29818bfc70a0,Старший официант,"ДВЕ ПАЛОЧКИ, сеть ресторанов свободного полета",•\tПодготовка ресторана к открытию;\r •\tВстре...,2008-12,2011-02
...,...,...,...,...,...,...
99995,df2f6158-c03b-4545-a4c5-e5bf80fd5a5b,Администратор ресторана,"Евразия, сеть ресторанов и суши-баров",Контроль работы ресторана,2014-06,2014-12
99996,df2f6158-c03b-4545-a4c5-e5bf80fd5a5b,Официант,jazz bar 48 chairs,Обслуживание гостей по всем европейским станда...,2011-06,2014-06
99997,df2f6158-c03b-4545-a4c5-e5bf80fd5a5b,"Суши-повар, официант, администратор",ООО Страйк,На поседней должности-контроль работы ресторана.,2007-04,2011-05
99998,7b0b9de2-d04c-42d5-a59d-6808b5abef8a,Член бригады ресторана,Макдоналдс,"Кассир, повар, грузчик.",2013-10,2014-11


In [4]:
display(vacancies)

,id,name,area.name,city,company.id,company,company_link,publication_date,salary_from,salary_currency,employment.name,schedule.name,experience.name,key_skills,specializations,description
0,17016760,Официант,Санкт-Петербург,Санкт-Петербург,1905558,2Т,https://hh.ru/employer/1905558,2016-06-16T11:22:58+0300,30000.0,RUR,Полная занятость,Полный день,Нет опыта,NaN,22.193 15.313,"Мы ищем активного, жизнерадостного и дружелюбн..."
1,17759717,Бармен-официант,Санкт-Петербург,unknown,2440865,3Ф Групп,https://hh.ru/employer/2440865,2016-07-15T13:00:14+0300,30000.0,RUR,Полная занятость,Полный день,Нет опыта,Ответственность | Материальная ответственность...,22.175 22.193,Обязанности: Приготовление напитков в баре ...
2,17816582,Администратор ресторана,Санкт-Петербург,Санкт-Петербург,1911500,4ЕС,https://hh.ru/employer/1911500,2016-07-20T17:20:48+0300,45000.0,RUR,Полная занятость,Сменный график,От 3 до 6 лет,NaN,22.329,"Ресторан ""Рандеву"" приглашает на работу админи..."
3,17746005,Кладовщик,Санкт-Петербург,посёлок Шушары,141100,4Ф,https://hh.ru/employer/141100,2016-08-13T09:26:06+0300,38000.0,RUR,Полная занятость,Сменный график,От 1 года до 3 лет,NaN,21.275 21.563,Обязанности: Прием товара (Кока-Кола) на скла...
4,17431972,Кладовщик,Санкт-Петербург,поселок Шушары,141100,4Ф,https://hh.ru/employer/141100,2016-07-12T15:22:23+0300,30000.0,RUR,Полная занятость,Полный день,От 1 года до 3 лет,NaN,21.563 21.564,Обязанности: отпуск и прием на складе ТМЦ вед...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32160,26344256,Кладовщик,Санкт-Петербург,Санкт-Петербург,3414,"ТрансМашХолдинг, Группа компаний",https://hh.ru/employer/3414,2018-06-13T19:25:26+0300,NaN,NaN,Полная занятость,Полный день,От 1 года до 3 лет,NaN,21.563,"ОАО ""Октябрьский электровагоноремонтный завод""..."
32161,26367364,Кондитер в ресторан (частичная занятость),Санкт-Петербург,unknown,3064104,ТРИГГЕР,https://hh.ru/employer/3064104,2018-06-14T17:56:22+0300,29000.0,RUR,Частичная занятость,Сменный график,От 1 года до 3 лет,NaN,22.204 22.353,"Ресторан ""Счастье у Исаакия"" открывает конкурс..."
32162,26373024,Заведующий производством/заведующий продовольс...,Санкт-Петербург,Пушкин,2130686,ФГБУЗ Санкт-Петербургский Дом-пансионат ветера...,https://hh.ru/employer/2130686,2018-06-15T09:56:47+0300,30000.0,RUR,Полная занятость,Полный день,От 1 года до 3 лет,NaN,22.204 18.297,Обязанности: - Руководство деятельностью пищеб...
32163,26431613,Грузчик-комплектовщик,Санкт-Петербург,unknown,66239,"Элизиум, группа компаний",https://hh.ru/employer/66239,2018-06-19T10:45:48+0300,30000.0,RUR,Полная занятость,Полный день,Нет опыта,NaN,29.495 29.545 21.275 21.564 21.136,Обязанности: Перемещение готовой продукции По...


# Analyzing data

In [5]:
klad = vacancies[vacancies["name"] == "Кладовщик"]

In [6]:
klad[klad["description"].str.contains("задачи", flags=re.IGNORECASE)]

,id,name,area.name,city,company.id,company,company_link,publication_date,salary_from,salary_currency,employment.name,schedule.name,experience.name,key_skills,specializations,description
290,17719816,Кладовщик,Санкт-Петербург,unknown,1863695,JACOBS DOUWE EGBERTS (JDE),https://hh.ru/employer/1863695,2016-07-12T13:48:40+0300,NaN,NaN,Полная занятость,Полный день,От 3 до 6 лет,Складской документооборот | Складской Учет | И...,21.563 21.564,Основные задачи: осуществлять руководство раб...
492,16327708,Кладовщик,Санкт-Петербург,unknown,14208,Акваарт,https://hh.ru/employer/14208,2016-08-01T10:38:27+0300,36000.0,RUR,Полная занятость,Полный день,От 1 года до 3 лет,NaN,21.563,Крупной компании требуется Кладовщик. Мы увлеч...
655,17647889,Кладовщик,Санкт-Петербург,unknown,681883,Велесстрой,https://hh.ru/employer/681883,2016-11-24T10:55:21+0300,45000.0,RUR,Полная занятость,Вахтовый метод,От 1 года до 3 лет,MS Office | 1С: Бухгалтерия и склад | 1С: Упра...,21.563 21.564,"ООО ""Велесстрой"" приглашает на работу Кладовщи..."
728,17909436,Кладовщик,Санкт-Петербург,unknown,1978,Группа компаний ТЕЛРОС,https://hh.ru/employer/1978,2016-08-01T12:55:15+0300,NaN,NaN,Полная занятость,Полный день,От 3 до 6 лет,NaN,21.563 21.564,Крупная телекоммуникационная компания пригла...
729,17775765,Кладовщик,Санкт-Петербург,unknown,1978,Группа компаний ТЕЛРОС,https://hh.ru/employer/1978,2016-07-18T11:29:01+0300,NaN,NaN,Полная занятость,Полный день,От 3 до 6 лет,NaN,21.563 21.564,Крупная телекоммуникационная компания пригла...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30446,14390895,Кладовщик,Санкт-Петербург,Санкт-Петербург,566086,Десерт Фентези,https://hh.ru/employer/566086,2015-08-24T13:28:33+0300,40000.0,RUR,Полная занятость,Сменный график,От 1 года до 3 лет,NaN,21.563 21.564,Должностные обязанности: Работа на складе зам...
30766,14088764,Кладовщик,Санкт-Петербург,Санкт-Петербург,24837,НГ-Энерго,https://hh.ru/employer/24837,2015-08-18T09:16:24+0300,NaN,RUR,Полная занятость,Полный день,От 1 года до 3 лет,NaN,21.563 21.564,"Компания ""НГ-Энерго"" приглашает в команду проф..."
31918,19907730,Кладовщик,Санкт-Петербург,unknown,837503,Сварочные Технологии,https://hh.ru/employer/837503,2017-03-13T10:00:59+0300,30000.0,RUR,Полная занятость,Полный день,Нет опыта,Деловая коммуникация | Организаторские навыки ...,21.176 21.275 21.563 21.564 21.136 15.308,Добрый день! Мы - интернет-магазин промышлен...
32007,18856837,Кладовщик,Санкт-Петербург,unknown,197216,ГК «РОСОХРАНА»,https://hh.ru/employer/197216,2016-11-09T17:12:55+0300,NaN,NaN,Проектная работа,Полный день,От 3 до 6 лет,NaN,21.275 21.563,Что мы ждем от кандидата: Знание порядка орга...


# Manually tagging

In [7]:
pd.set_option('display.max_colwidth', None)

In [8]:
ids = [14829806,
       25374959,
       18985243,
       22011507,
       15945853,
       25845683,
       19176449,
       19852081,
       22681923,
       19899949]

In [9]:
vacancies[vacancies['id'].isin(ids)]

,id,name,area.name,city,company.id,company,company_link,publication_date,salary_from,salary_currency,employment.name,schedule.name,experience.name,key_skills,specializations,description
1799,19899949,Кладовщик,Санкт-Петербург,unknown,1947314,ANCOR Consumer Goods,https://hh.ru/employer/1947314,2017-04-04T17:34:37+0300,50000.0,RUR,Полная занятость,Полный день,От 1 года до 3 лет,NaN,21.563,"Наш клиент, крупная торговая компания, открывает вакансию Кладовщик. Обязанности: Руководство складом пищевой продукции (кондитерские изделия) - хранение, погрузка, разгрузка, сбор заказов, формирование заказов по маршрутам и машинам доставки; Организация и контроль бесперебойной работы склада; Руководство персоналом (грузчики); Отчетность. Требования: Высшее образование; Опыт работы на аналогичной должности от 2-х лет; Опыт работы на складе пищевой продукции обязателен; Опытный пользователь Microsoft Office, 1C. Условия: Оформление в соответствии с ТК РФ Работа на юге города"
8580,25374959,Кладовщик,Санкт-Петербург,Москва,600589,Нью Йоркер РУС,https://hh.ru/employer/600589,2018-04-13T09:40:38+0300,NaN,NaN,Полная занятость,Полный день,Нет опыта,NaN,21.563 21.564,"Обязанности: Размещение и хранение товара на складе Учет товарных остатков Проведение инвентаризации Погрузочно-разгрузочные работы Контроль своевременного пополнения товарного запаса в торговом зале Распределение товара на складе Выдача товара в торговый зал Поддержание чистоты и порядка на складе Работа с товаром по предпродажной подготовке (распаковка товара, оформление ценников, развеска товара) Требования: Опыт работы в сфере розничной торговли товарами народного потребления приветствуется Законченное среднее образование Условия: Официальное трудоустройство с соблюдением всех норм ТК РФ Достойная заработная плата и гибкая система премирования по результатам продаж Молодой и дружный коллектив Возможность повышения квалификации ( внутреннее и внешнее обучение) и карьерного роста Полная занятость, полный день Сменный график работы Работа В ТЦ ""Лето"""
9768,22011507,Кладовщик,Санкт-Петербург,поселок Шушары,7348,SMC Pneumatik,https://hh.ru/employer/7348,2017-08-02T13:06:03+0300,NaN,NaN,Полная занятость,Полный день,От 1 года до 3 лет,NaN,21.563 21.564,"Обязанности: Разбор поставок, сборка, упаковка и подготовка заказов к отправке. Требования: Аналогичный опыт работы на складе от 1 года; Внимательность, аккуратность, ответственность. Условия: Оформление в соответствии с ТК РФ, ""белая"" заработная плата; График работы: 5/2, полный рабочий день с 9.00 до 18.00; Место работы: теплый склад, п. Шушары, 2-й Бадаевский проезд; Развозка или проезд на любой маршрутке от ст.м.""Купчино"", с Витебского пр. до магазина ""Лента""; Социальный пакет (ДМС, НС, доп. отпуск, 100% оплата больничных листов)."
11782,18985243,Кладовщик,Санкт-Петербург,unknown,872828,"БАЛТИЙСКИЙ БЕРЕГ, ТД",https://hh.ru/employer/872828,2017-01-12T17:41:29+0300,40000.0,RUR,Полная занятость,Сменный график,От 1 года до 3 лет,NaN,21.275 21.563,"Компания ""БАЛТИЙСКИЙ БЕРЕГ""- лидер на рынке по производству и реализации рыбной продукции, приглашает в свою команду кладовщика. Обязанности: Приёмка и отгрузка товара со склада; Оформление складских документов (приходные, расходные ордера); Инвентаризация товара на складе; Работа с терминалом сбора данных. Требования: Среднее-специальное или среднее-техническое образование; Опыт работы от двух лет; Знание 1С 8 в конфигурациях УТ или УПП; Наличие действующей медицинской книжки. Мы предлагаем: Работу в стабильной, динамично развивающейся компании; Оформление согласно ТК РФ; Официальную заработную плату 40 000 рублей; График работы сменный день/ночь; Дружный, молодой коллектив; Место работы Горское шоссе (Комплекс Осиновая роща). Развозка от ст. м. Озерки."
12314,19176449,Кладовщик,Санкт-Петербург,unknown,85975,САКС,https://hh.ru/employer/85975,2016-12-12T09:51:57+0300,25000.0,RUR,Полная занятость,Полный день,От 1 года до 3 лет,NaN,21.563 21.564,"Обязанности: прием хранение выдача, отгру

# Splitting description

In [10]:
# Tags by which we split description.
tags = {
    "Обязанности": ["обязан", "задачи"],
    "Требования": ["требуется", "требовани", "пожелан"],
    "Условия": ["условия", "предлагаем", "предложени"]
}

# Tags which we exclude.
bad_tags = {
    "Обязанности": [],
    "Требования": ["требований", "требованиям", "требования безо", "требования по"],
    "Условия": ["условиями"]
}

In [11]:
def get_data_from_description(description, tags, bad_tags):
    """Extracting text by key words.
    
    Args:
        description (string): text we want to extract
                              tags from.
        tags (dict): keys are tags we want to extract;
                     values are lists of words that
                         match this tag.
    
    Returns:
        data_from_description (dict): 
                     keys are tags;
                     values are text that match key.
    """
    
    # Merging all keywords from tags to 1 list.
    keywords = reduce(lambda x, y: x + y, list(tags.values()))
    bad_keywords = reduce(lambda x, y: x + y, list(bad_tags.values()))
    
    # Splitting description by keywords. 
    part = []
    all_parts = []
    for word in description.split():
        if any(subword in word.lower() for subword in keywords) and (not (any(bad_subword in word.lower() for bad_subword in bad_keywords))):
            all_parts.append(part)
            all_parts.append([word])
            part = []
        else:
            part.append(word)
    all_parts.append(part)

    # Merging lists of strings to strings.
    all_strings = []
    for word_list in all_parts:
        all_strings.append(' '.join(word_list))

    # Creating dictionary with keys same as tags keys and
    # values as all strings matching these keys
    # (value is a text that follows current key)
    data_from_description = {}
    for string in all_strings:
        for key in tags.keys():
            if any(subword in string.lower() for subword in tags[key]) and (not (any(bad_subword in string.lower() for bad_subword in bad_keywords))):
                data_from_description[key] = all_strings[all_strings.index(string) + 1]

    return all_strings, data_from_description

## testing

In [12]:
with open("./prepared_data/storekeeper_labeled.json", "r") as read_file:
    data = json.load(read_file)

In [13]:
print("Количество ключей: " + str(len(data.keys())))

Количество ключей: 28


In [14]:
for id in list(data.keys()):
    print("Id:")
    print(id)
    print("\n")
    
    print("Human processed data:")
    print(json.dumps(data[id], indent=4, ensure_ascii=False))
    print("\n")
    
    description = list(vacancies[vacancies["id"] == int(id)]["description"])[0]
    all_strings, data_from_description = get_data_from_description(description, tags, bad_tags)
    
    print("Parsed data:")
    print(json.dumps(data_from_description, indent=4, sort_keys=True, ensure_ascii=False))
    print("\n")
    
    print("Meta data:")
    print(all_strings)
    print("\n")
    
    print("-------------------------------------------------------------------------")

Id:
25374959


Human processed data:
{
    "common": {
        "q0": "Кладовщик",
        "q1": "Санкт-Петербург",
        "q2": "Московский район",
        "q3": "Нью Йоркер РУС",
        "q4": "",
        "q5": [
            "Полная занятость"
        ],
        "q6": [
            "Полный день"
        ],
        "q7": "нет опыта",
        "q8": 730,
        "q9": [
            "опыт работы в сфере розничной торговли товарами народного потребления",
            "приветствуется законченное среднее образование"
        ],
        "q10": [
            "Размещение и хранение товара на складе",
            "Учет товарных остатков",
            "Проведение инвентаризации ",
            "Погрузочно-разгрузочные работы",
            "Контроль своевременного пополнения товарного запаса в торговом зале",
            "Распределение товара на складе ",
            "Выдача товара в торговый зал ",
            "Поддержание чистоты и порядка на складе ",
            "Работа с товаром по предпродаж

In [15]:
description = list(vacancies[vacancies["id"] == 19899949]["description"])[0]

In [16]:
vacancies[vacancies["id"] == 19899949]

,id,name,area.name,city,company.id,company,company_link,publication_date,salary_from,salary_currency,employment.name,schedule.name,experience.name,key_skills,specializations,description
1799,19899949,Кладовщик,Санкт-Петербург,unknown,1947314,ANCOR Consumer Goods,https://hh.ru/employer/1947314,2017-04-04T17:34:37+0300,50000.0,RUR,Полная занятость,Полный день,От 1 года до 3 лет,NaN,21.563,"Наш клиент, крупная торговая компания, открывает вакансию Кладовщик. Обязанности: Руководство складом пищевой продукции (кондитерские изделия) - хранение, погрузка, разгрузка, сбор заказов, формирование заказов по маршрутам и машинам доставки; Организация и контроль бесперебойной работы склада; Руководство персоналом (грузчики); Отчетность. Требования: Высшее образование; Опыт работы на аналогичной должности от 2-х лет; Опыт работы на складе пищевой продукции обязателен; Опытный пользователь Microsoft Office, 1C. Условия: Оформление в соответствии с ТК РФ Работа на юге города"
